In [0]:
%run /Workspace/Users/valterlafuentejunior@gmail.com/ProjetoMedalhao/00.config/config

In [0]:
from pyspark.sql.functions import current_date
import pandas as pd
from delta.tables import DeltaTable


In [0]:
tabela_destino = "workspace.bronze_etl.boi_gordo"



In [0]:
df = (
    spark.read
    .option("header", "true") # cabeçalho
    .option("sep", ";")   
    .option("inferSchema", "true") # inferir tipos automaticamente
    .csv("/Volumes/workspace/bronze_etl/boigordo/BoiGordo.csv")
)



In [0]:
df = df.withColumn("data_coleta", current_date())

In [0]:
# Merge delta lake
if df is not None and df.limit(1).count() > 0:
    print(f"Total de registros carregados: {df.count()}")

    if spark.catalog.tableExists(tabela_destino):
        print(f"Tabela {tabela_destino} já existe — atualizando dados...")
        delta_table = DeltaTable.forName(spark, tabela_destino)
        
        (
            delta_table.alias("t")
            .merge(
                df.alias("s"),
                "t.data_coleta = s.data_coleta AND t.data = s.data"
            )
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )
    else:
        print(f"Tabela {tabela_destino} não existe — criando nova tabela...")
        df.write.format("delta").partitionBy("data_coleta").mode("append").saveAsTable(tabela_destino)
else:
    print("Nenhum dado para atualizar.")       